In [ ]:
from decimal import Decimal
import pickle
from google.cloud import language
import numpy as np
import pandas as pd

### USAGE NOTES
* Ensure comments.csv, read_csv, and to_csv all using UTF-8
* Set const LOAD_FROM_PICKLE (False if e.g. changing what funcs return)

In [ ]:
LOAD_FROM_PICKLE = False
# Ignore categorical questions stored in same table
IGNORE_LIST = ['Prep']

In [ ]:
# Load dataset
df = pd.read_csv('data/comments.csv',
                 sep=',',
                 header=0,
                 encoding='utf-8',      # UTF-8 for FR
                 keep_default_na=False, # Prevent empty strings from being converted to NaN
                 nrows=10)

In [ ]:
# Fix French chars with ANSI encodings displayed as UTF-8
char_dict = {
    'Ã§': 'ç',
    'Ã‡': 'Ç',
    'Ã©': 'é',
    'Ã‰': 'É',
    'Ã ': 'à',
    'Ã¨': 'è',
    'Ã¬': 'ì',
    'Ã²': 'ò',
    'Ã¹': 'ù',
    'Ã€': 'À',
    'Ãˆ': 'È',
    'ÃŒ': 'Ì',
    'Ã’': 'Ò',
    'Ã™': 'Ù',
    'Ã¢': 'â',
    'Ãª': 'ê',
    'Ã®': 'î',
    'Ã´': 'ô',
    'Ã»': 'û',
    'Ã‚': 'Â',
    'ÃŠ': 'Ê',
    'ÃŽ': 'Î',
    'Ã”': 'Ô',
    'Ã›': 'Û',
    'Ã«': 'ë',
    'Ã¯': 'ï',
    'Ã¼': 'ü',
    'Ã‹': 'Ë',
    'Ã': 'Ï', # Yes, second char is non-displaying
    'Ãœ': 'Ü'
}
df.replace(to_replace=char_dict, value=None, inplace=True, regex=True)

In [ ]:
df.head()

In [ ]:
# Load pickle for memoization
if LOAD_FROM_PICKLE:
    with open('memo.pickle', 'rb') as f:
        memo_dict = pickle.load(f)
else:
    memo_dict = {}

In [ ]:
# Instantiate client
client = language.LanguageServiceClient()

# Counter to track progress with large datasets
ctr = 0

def analyze_text(survey_id, short_question, my_text):
    """Pass sample to API and return tuple of shape (sentiment, magnitude).
    Memoize results using 'survey_id' as PKEY."""
    # Print ctr for every 1000 comments
    global ctr
    ctr += 1
    if ctr % 1000 == 0:
        print('Finished {0} comments!'.format(i))
    
    if short_question in IGNORE_LIST:
        result = (np.nan, np.nan)
        # Memoize and return result
        memo_dict[survey_id] = result
        return result
    
    # If already processed, returned memoized result to save compute
    if survey_id in memo_dict:
        return memo_dict[survey_id]
    
    # Pass to API
    document = language.types.Document(content=my_text,
                                       type=language.enums.Document.Type.PLAIN_TEXT)
    sentiment = client.analyze_sentiment(document=document).document_sentiment
    # Adjust interval from [-1, 1] to [1, 5]
    # Cast to Decimal then back to int to prevent floating point rounding errors
    sentiment_score = int(round(Decimal(str((sentiment.score * 2) + 3))))
    magnitude = sentiment.magnitude
    result = (sentiment_score, magnitude)
    # Memoize and return result
    memo_dict[survey_id] = result
    return result

In [ ]:
%%time
api_results = df.apply(lambda x: analyze_text(x['text_answer'], x['survey_id']),
                       axis=1,               # Apply to each row
                       raw=False,            # Pass each cell individually as not using NumPy
                       result_type='expand') # Return DataFrame rather than Series of tuples

In [ ]:
df['stars'] = api_results[0]
df['magnitude'] = api_results[1]

In [ ]:
df.head()

In [ ]:
# Export memo_dict to pickle for future re-use
with open('memo.pickle', 'wb') as f:
    pickle.dump(memo_dict, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Export CSV
df.to_csv('data/comments_ML.csv', sep=',', index=False, encoding='utf-8')